# Importing

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

In [2]:
data = pd.read_csv('cardio_train.csv',sep=";")

# converting age

In [3]:
# .25 is because every 4 years is a leap year
def convertAge(data):
    data['age'] = data['age'] / 365.25
    data = data.drop_duplicates()
    return data
data = convertAge(data)
# lets drop the ID column
data = data.iloc[::,1::]

# 6. Outlier Detection & Treatment




(complete research on outliers in in the Obsedian app)


## Identify outliers using:

Boxplots



IQR method


In [4]:
def IQR(data):
    # Convert input to numpy array if it's a list
    data = np.array(data)

    # Calculate the first and third quartiles (Q1 and Q3)
    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)

    # Calculate the Interquartile Range (IQR)
    IQR = Q3 - Q1

    # Define the lower and upper bounds
    # The standard multiplier is 1.5
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Identify outliers (points outside the bounds)
    outliers = [x for x in data if x < lower_bound or x > upper_bound]
    total_out = len(outliers)
    return {
        'length' : total_out,
        'list' : outliers
    }
sum=0
# for each column in data (dataset)
for i in data.columns:
    print("THE column : ",i)
    dict = IQR(data[i])
    sum += dict['length']
    if dict['length']!=0:
        print("the outliers length in i",dict['length'])
        print("the list of ouliers in i",dict['list'])
print("total outliers found using IQR method : ",sum)

THE column :  age
the outliers length in i 4
the list of ouliers in i [np.float64(29.782340862423), np.float64(29.56331279945243), np.float64(29.730321697467488), np.float64(30.017796030116358)]
THE column :  gender
THE column :  height
the outliers length in i 519
the list of ouliers in i [np.int64(187), np.int64(188), np.int64(76), np.int64(142), np.int64(141), np.int64(195), np.int64(140), np.int64(198), np.int64(187), np.int64(196), np.int64(187), np.int64(138), np.int64(187), np.int64(142), np.int64(194), np.int64(190), np.int64(134), np.int64(140), np.int64(136), np.int64(195), np.int64(198), np.int64(187), np.int64(100), np.int64(190), np.int64(190), np.int64(120), np.int64(120), np.int64(187), np.int64(120), np.int64(189), np.int64(189), np.int64(187), np.int64(194), np.int64(189), np.int64(196), np.int64(137), np.int64(190), np.int64(140), np.int64(187), np.int64(189), np.int64(188), np.int64(187), np.int64(187), np.int64(137), np.int64(192), np.int64(187), np.int64(196), np.i


Z-score method


In [5]:
def zscore(data,threshold=3):
    mean = np.mean(data)
    std = np.std(data)
    z_scores = [(i - mean) / std for i in data]
    
    outliers = []
    outlier_indices = []
    for i, z_score in enumerate(z_scores):
        if np.abs(z_score) > threshold:
            outliers.append(data[i])
            outlier_indices.append(i)
            
    return {
        'outliers': outliers,
        'outlier_indices': outlier_indices,
        'z_scores': np.array(z_scores),
        'length' : len(outlier_indices)
    }
total = 0
for i in data.columns:
    results = zscore(data[i],threshold=3)
    total += results['length']
    if results['length'] !=0 :
        print(f"Outliers: {results['outliers']}")
        print(f"Outlier Indices: {results['outlier_indices']}")
        print(f"length: {results['length']}")
print("the total outliers after zscore : ",total)


Outliers: [np.float64(29.782340862423), np.float64(29.56331279945243), np.float64(29.730321697467488), np.float64(30.017796030116358)]
Outlier Indices: [6219, 22343, 30666, 55905]
length: 4
Outliers: [np.int64(76), np.int64(195), np.int64(198), np.int64(196), np.int64(138), np.int64(194), np.int64(190), np.int64(134), np.int64(136), np.int64(195), np.int64(198), np.int64(100), np.int64(190), np.int64(190), np.int64(120), np.int64(120), np.int64(120), np.int64(189), np.int64(189), np.int64(194), np.int64(189), np.int64(196), np.int64(137), np.int64(190), np.int64(189), np.int64(137), np.int64(192), np.int64(196), np.int64(122), np.int64(190), np.int64(250), np.int64(191), np.int64(190), np.int64(120), np.int64(189), np.int64(117), np.int64(120), np.int64(70), np.int64(97), np.int64(190), np.int64(119), np.int64(137), np.int64(198), np.int64(120), np.int64(120), np.int64(130), np.int64(192), np.int64(136), np.int64(130), np.int64(190), np.int64(120), np.int64(110), np.int64(193), np.int6

the difference between total outliers by IQR and zscore method

In [6]:
diff = sum - total
print("the difference : ",abs(diff))
if sum > total:
    print("zscore is better")
else :
    print("IQR is better")

the difference :  25350
zscore is better


In [7]:
def impossible(data):
    condition = (
        (data['age'] < 0) | (data['age'] > 120) |
        (data['height'] <= 0) | (data['height'] > 250) |
        (data['weight'] <= 0) | (data['weight'] > 250) |
        (data['ap_hi'] < 50) | (data['ap_hi'] > 300) |
        (data['ap_lo'] < 30) | (data['ap_lo'] > 200) |
        (data['ap_hi'] <= data['ap_lo'])
    )
    return data[condition]

out_df = impossible(data)
out_df
print("Invalid age:", ((data['age'] < 0) | (data['age'] > 120)).sum())
print("Invalid BP relation:", (data['ap_hi'] <= data['ap_lo']).sum())
print("Invalid height:", ((data['height'] <= 0) | (data['height'] > 250)).sum())
print("Invalid weight:", ((data['weight'] <= 0) | (data['weight'] > 250)).sum())
print("Invalid ap_hi:", ((data['ap_hi'] < 50) | (data['ap_hi'] > 300)).sum())
print("Invalid ap_lo:",  ((data['ap_lo'] < 30) | (data['ap_lo'] > 200)).sum())
print("total impossible value is",out_df.shape[0])


Invalid age: 0
Invalid BP relation: 1236
Invalid height: 0
Invalid weight: 0
Invalid ap_hi: 228
Invalid ap_lo: 1006
total impossible value is 1322


In [8]:
data.columns

Index(['age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol',
       'gluc', 'smoke', 'alco', 'active', 'cardio'],
      dtype='object')


## Handle outliers:

Remove them


In [9]:
# remove all the impossible value's records
data = data[data['ap_hi'] > data['ap_lo']]



Cap/floor them


In [10]:
# clip extreme values into specific range
data['height'] = data['height'].clip(120, 220)
data['weight'] = data['weight'].clip(30, 200)
data['ap_hi'] = data['ap_hi'].clip(90, 240)
data['ap_lo'] = data['ap_lo'].clip(60, 140)


In [11]:
data.shape

(68764, 12)

In [12]:
# exporting this dataset into new one
data.to_csv('processed_cardio_train.csv',index=False)


Transform data (log, sqrt, etc.)

# 7. Feature Engineering / Transformation

Encode categorical variables:

Label Encoding

One-Hot Encoding

Create new features from existing ones

Scale/normalize numerical features:

StandardScaler, MinMaxScaler, RobustScaler

Log / Box-Cox transformations for skewed features

Binning continuous variables into categories

# 8. Checking Relationships

Correlation:

Pearson (numerical)

Spearman (ordinal)

Group-wise aggregation & visualization

Crosstab and pivot tables

# 9. Checking Data Quality

Check for inconsistent values (e.g., typos in categorical columns)

Check for impossible values (e.g., negative ages)

Check distribution of target variable (for supervised ML)

# 10. Summary & Insights

Generate a summary report:

Missing values

Outliers

Key correlations

Feature importance (optional)

Identify potential features for modeling

Decide next steps for preprocessing